<a href="https://colab.research.google.com/github/drbronson/Auto-GPT/blob/master/Scripts_to_Generate_Tableau_Migration_CSV_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import subprocess
import os
import sys
import getpass # For securely getting password input

# =====================================================================================================================
# Script 1: generate_new_users_template.py
# Purpose: Generates a template for 'new_eads_users_to_add.csv' based on a provided list of new EADS usernames.
#          You'd typically get this list from your EADS/ID team or derive it from your user mapping.
# =====================================================================================================================

def generate_new_users_template(output_filename="new_eads_users_to_add.csv", eads_users_list=None):
    """
    Generates a CSV template for adding new EADS users to Tableau Server.

    Args:
        output_filename (str): The name of the CSV file to create.
        eads_users_list (list): A list of new EADS usernames (e.g., ["EADS_DOMAIN\\john.doe", ...]).
                                If None, it will prompt for input.
    """
    print(f"\n--- Generating {output_filename} template ---")
    if eads_users_list is None:
        print("Enter new EADS usernames (one per line). Type 'done' when finished:")
        temp_list = []
        while True:
            user_input = input("> ").strip()
            if user_input.lower() == 'done':
                break
            if user_input:
                temp_list.append(user_input)
        eads_users_list = temp_list

    if not eads_users_list:
        print("No EADS users provided. Skipping file generation.")
        return

    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['username', 'password', 'site_name', 'role']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for username in eads_users_list:
            # Assign dummy password and default site/role.
            # You'll likely need to manually adjust roles and actual sites.
            # If using SSO, passwords might not be strictly needed for tabcmd createusers (use --no-password).
            writer.writerow({
                'username': username,
                'password': 'ChangeMe123!', # Placeholder, change securely or omit if using --no-password
                'site_name': 'Default',    # Adjust as needed per user
                'role': 'Viewer'           # Adjust as needed per user
            })
    print(f"'{output_filename}' generated. Review and manually adjust passwords, site_name, and role columns.")

# =====================================================================================================================
# Script 2: generate_user_mapping_template.py
# Purpose: Connects to the OLD Tableau Server (via tabcmd) to list content and its current owners.
#          Generates a template for 'user_mapping.csv', leaving 'new_username' column blank for manual completion.
#
# Prerequisites:
# - tabcmd must be installed on the machine running this script.
# - tabcmd must be logged into the OLD Tableau Server.
# =====================================================================================================================

def generate_user_mapping_template(old_tableau_server_url, old_admin_username, output_filename="user_mapping.csv"):
    """
    Generates a template for 'user_mapping.csv' by listing content and owners from the old Tableau Server.
    Requires tabcmd to be logged into the old server.
    """
    print(f"\n--- Generating {output_filename} template from OLD Tableau Server ---")
    print(f"Please ensure tabcmd is logged into your OLD Tableau Server ({old_tableau_server_url}) as '{old_admin_username}'")
    print("Example login command: tabcmd login -s <old_url> -u <username> -p <password> --site Default")
    print("Fetching list of sites...")

    # Get list of sites on the old server
    sites = []
    try:
        result = subprocess.run(["tabcmd", "listsites", "--no-cookie"], capture_output=True, text=True, check=True)
        # Parse output (simple regex might be needed for robust parsing)
        # Assuming output format like: "Site Name        Content URL"
        for line in result.stdout.splitlines():
            if line and "Site Name" not in line and "---" not in line and "=======" not in line:
                parts = line.split()
                if len(parts) >= 2:
                    sites.append(parts[1]) # Content URL
        print(f"Found sites on old server: {sites}")
    except subprocess.CalledProcessError as e:
        print(f"Error listing sites from old server: {e.stderr}")
        print("Please ensure tabcmd is logged in and accessible.")
        return

    content_data = []

    for site_name in sites:
        print(f"Processing site: {site_name}")
        site_param = "--site " + site_name if site_name != "Default" else "" # Use --site Default for default site

        # List Workbooks and their owners (requires parsing tabcmd listworkbooks output)
        try:
            workbook_list_cmd = ["tabcmd", "listworkbooks"]
            if site_name != "Default":
                workbook_list_cmd.extend(["--site", site_name])
            workbook_list_cmd.append("--no-cookie") # Use --no-cookie for batch processing

            result = subprocess.run(workbook_list_cmd, capture_output=True, text=True, check=True)
            for line in result.stdout.splitlines():
                # Example line: "Workbook1  ProjectA  john.doe  2023-01-01  Published"
                parts = line.split("  ") # Tabcmd output is often space-delimited
                if len(parts) >= 3 and "Name" not in parts[0] and "---" not in parts[0]:
                    workbook_name = parts[0].strip()
                    owner_name = parts[2].strip() # This might be just username, not domain\username
                    # Attempt to get domain for owner if not present (heuristic)
                    if "\\" not in owner_name:
                         # This is a heuristic. You MUST confirm the old domain prefix.
                        owner_name = f"OLD_DOMAIN\\{owner_name}"
                    content_data.append({
                        'content_type': 'Workbook',
                        'content_name': workbook_name,
                        'site_name': site_name,
                        'old_username': owner_name,
                        'new_username': '' # To be filled manually
                    })
        except subprocess.CalledProcessError as e:
            print(f"  Error listing workbooks for site {site_name}: {e.stderr}")

        # List Data Sources and their owners
        try:
            datasource_list_cmd = ["tabcmd", "listdatasources"]
            if site_name != "Default":
                datasource_list_cmd.extend(["--site", site_name])
            datasource_list_cmd.append("--no-cookie")

            result = subprocess.run(datasource_list_cmd, capture_output=True, text=True, check=True)
            for line in result.stdout.splitlines():
                # Example line: "DataSource1  ProjectA  jane.smith  2023-01-01  Published"
                parts = line.split("  ")
                if len(parts) >= 3 and "Name" not in parts[0] and "---" not in parts[0]:
                    datasource_name = parts[0].strip()
                    owner_name = parts[2].strip()
                    if "\\" not in owner_name:
                        owner_name = f"OLD_DOMAIN\\{owner_name}" # Heuristic
                    content_data.append({
                        'content_type': 'Datasource',
                        'content_name': datasource_name,
                        'site_name': site_name,
                        'old_username': owner_name,
                        'new_username': '' # To be filled manually
                    })
        except subprocess.CalledProcessError as e:
            print(f"  Error listing data sources for site {site_name}: {e.stderr}")

        # NOTE: tabcmd does not easily list flow owners. This would require Tableau REST API of OLD server.
        # For Flows, you might need manual collection or an assumption of owner.
        # For Projects, ownership is usually the creator. Can be added manually if needed.

    if not content_data:
        print("No content found or errors occurred. User mapping file will be empty.")
        return

    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['content_type', 'content_name', 'site_name', 'old_username', 'new_username']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(content_data)
    print(f"'{output_filename}' generated. **CRITICAL: Manually populate the 'new_username' column with the corresponding EADS users.**")

# =====================================================================================================================
# Script 3: generate_permission_mapping_template.py
# Purpose: Generates a template for 'permission_mapping.csv'.
#          This script creates a *sample* structure and does NOT extract existing permissions from the old server.
#          It provides placeholders for you to manually specify permissions for EADS users/groups.
#
# This is because extracting explicit permissions from the OLD Tableau Server (especially at scale)
# is best done via its REST API, which is a separate, more complex endeavor.
# =====================================================================================================================

def generate_permission_mapping_templa